In [84]:
%pip install tensorflow
%pip install seaborn
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [85]:
import os

#base settings

baseDir = "./datasets/SetA/"
os.listdir(baseDir)

batch_size = 4
image_size = 224

In [86]:
import tensorflow as tf

# Load training and validation datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    baseDir + "train",
    labels="inferred", #figure out the values
    label_mode="categorical",  # Use categorical for multi-class classification
    image_size=(image_size,image_size),
    batch_size=batch_size,
    shuffle=True,
    seed=42            
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    baseDir + "dev",
    labels="inferred",
    label_mode="categorical",
    image_size=(image_size,image_size),
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

# Test dataset (assuming separate directory for test data)
test_ds = tf.keras.utils.image_dataset_from_directory(
    baseDir + "test",
    labels="inferred",
    label_mode="categorical",
    image_size=(image_size,image_size),
    batch_size=batch_size,
    shuffle=False  # No shuffling for test set
)

print(train_ds.class_names)

Found 383 files belonging to 3 classes.
Found 109 files belonging to 3 classes.
Found 58 files belonging to 3 classes.
['Apple', 'Banana', 'Orange']


In [87]:
# define model layers
from tensorflow.keras import layers
import tensorflow as tf

pretrained_model = tf.keras.applications.MobileNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(image_size, image_size, 3),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

model = tf.keras.models.Sequential([
    pretrained_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# model = tf.keras.models.Sequential([
#     layers.Rescaling(1./255, input_shape=(image_size, image_size, 3)),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Conv2D(64, 3, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(3, activation='softmax')
# ])

In [88]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 1000)           │     3,538,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 1000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │       128,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,667,499 (13.99 MB)

 Trainable params: 3,633,387 (13.86 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [89]:
#compile model

for layer in model.layers:
    layer.trainable = False

for layer in model.layers[-15:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
                  loss='categorical_crossentropy',  # Suitable for classification
                  metrics=['accuracy'])

In [90]:
epochs = 20
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 58s 436ms/step - accuracy: 0.3723 - loss: 1.0986 - val_accuracy: 0.2202 - val_loss: 1.0992
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 60s 626ms/step - accuracy: 0.3934 - loss: 1.0986 - val_accuracy: 0.2661 - val_loss: 1.0987
Epoch 3/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 49s 512ms/step - accuracy: 0.3990 - loss: 1.0984 - val_accuracy: 0.2294 - val_loss: 1.0997
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 37s 387ms/step - accuracy: 0.3885 - loss: 1.0983 - val_accuracy: 0.2752 - val_loss: 1.0998
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 40s 416ms/step - accuracy: 0.4231 - loss: 1.0982 - val_accuracy: 0.2936 - val_loss: 1.0988
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 38s 395ms/step - accuracy: 0.4804 - loss: 1.0982 - val_accuracy: 0.3028 - val_loss: 1.0983
Epoch 7/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 39s 403ms/step - accuracy: 0.5482 - loss: 1.0979 - val_accuracy: 0.3945 - val_loss: 1.0977
Epoch 8/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 36s 372ms/step - accuracy: 0.5310 - loss: 1.0978 - val_accu

In [ ]:
model.save("./models/model.keras")

In [ ]:
import matplotlib.pyplot as plt

model.evaluate(test_ds)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Get predictions and true labels
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

y_true = []

for images, labels in test_ds:
    y_true.extend(np.argmax(labels.numpy(), axis=1))
y_true = np.array(y_true)

y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate precision, recall, f1-score (macro average)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
print(f"Precision: {precision:.3f}")
print(f"Recall (Sensitivity): {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

# Plot confusion matrix
class_names = train_ds.class_names
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
#prediction

import tensorflow as tf
import numpy as np

# === Settings ===
model_path = "./models/model.keras"            # Replace with your model path
image_path = "./images/test/orange_30.jpg"        # Replace with your image path
class_names = ["Apple", "Banana", "Orange"]  # Your class labels

model = tf.keras.models.load_model(model_path)

# === Load & Preprocess Image ===
img = tf.keras.utils.load_img(image_path, target_size=(image_size,image_size))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, axis=0)      # Add batch dimension

# === Predict ===
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
confidence = np.max(predictions[0])

# === Output ===
print(f"Predicted: {class_names[predicted_class]} ({confidence:.2f})")
